# Chapter 8 : Quizz generator

In [24]:
def write_header(fname, id) :
    fname.write("Name:\n")
    fname.write("Date:\n")
    fname.write("Period:\n")
    fname.write((" "*20) + "State Capitals Quiz (Form {0})\n\n".format(id))

def generate_question(state, capitals, noptions):
    import string
    # generate the list of options, randomly shuffled
    correct = capitals[state]
    wrong = list(capitals.values())
    wrong.remove(correct)
    answers = random.sample(wrong, noptions-1)
    answers += [correct]
    random.shuffle(answers)
    # write the question text
    question_text="The capital of {0} is:\n".format(state)
    for idoption in range(noptions) :
        question_text += "{0}. {1}\n".format(string.ascii_uppercase[idoption], answers[idoption])
    question_text += "\n"
    # write the solution text
    solution_text = "{0}. {1} -> {2}\n".format(state, 
                                                string.ascii_uppercase[answers.index(correct)],
                                                correct)
    
    return question_text, solution_text
    

In [26]:
import random

# quizz data
# The quiz data. Keys are states and values are their capitals.
capitals = {'Alabama': 'Montgomery', 'Alaska': 'Juneau', 'Arizona': 'Phoenix',
   'Arkansas': 'Little Rock', 'California': 'Sacramento', 'Colorado': 'Denver',
   'Connecticut': 'Hartford', 'Delaware': 'Dover', 'Florida': 'Tallahassee',
   'Georgia': 'Atlanta', 'Hawaii': 'Honolulu', 'Idaho': 'Boise', 'Illinois':
   'Springfield', 'Indiana': 'Indianapolis', 'Iowa': 'Des Moines', 'Kansas':
   'Topeka', 'Kentucky': 'Frankfort', 'Louisiana': 'Baton Rouge', 'Maine':
   'Augusta', 'Maryland': 'Annapolis', 'Massachusetts': 'Boston', 'Michigan':
   'Lansing', 'Minnesota': 'Saint Paul', 'Mississippi': 'Jackson', 'Missouri':
   'Jefferson City', 'Montana': 'Helena', 'Nebraska': 'Lincoln', 'Nevada':
   'Carson City', 'New Hampshire': 'Concord', 'New Jersey': 'Trenton', 
    'New Mexico': 'Santa Fe', 'New York': 'Albany', 'North Carolina': 'Raleigh',
   'North Dakota': 'Bismarck', 'Ohio': 'Columbus', 'Oklahoma': 'Oklahoma City',
   'Oregon': 'Salem', 'Pennsylvania': 'Harrisburg', 'Rhode Island': 'Providence',
   'South Carolina': 'Columbia', 'South Dakota': 'Pierre', 'Tennessee':
   'Nashville', 'Texas': 'Austin', 'Utah': 'Salt Lake City', 'Vermont':
   'Montpelier', 'Virginia': 'Richmond', 'Washington': 'Olympia', 
    'West Virginia': 'Charleston', 'Wisconsin': 'Madison', 'Wyoming': 'Cheyenne'}
             
# Define some parameters
NQUIZZ=10 # Number of quizes
NQUESTIONS=2 # Number of questions per quizz
NOPTIONS=4 # Four options per question

# Generate the quizzes
for idquizz in range(NQUIZZ) :
    print("Generating quizz number : {0}".format(idquizz))
    # TODO : Create the quizz and answer files.
    quizzfile = open("test/capitals-quizz-{0:02}.txt".format(idquizz+1), 'w')
    solfile = open("test/capitals-quizz-{0:02}-solutions.txt".format(idquizz+1), 'w')
    
    # TODO : Write out the header for the quiz. 
    write_header(quizzfile, idquizz)
    
    # TODO : Shuffle the order of the states.
    states = list(capitals.keys())
    random.shuffle(states)
    
    # TODO : Generate NQUESTIONS and write them to the file. 
    for idquestion in range(NQUESTIONS) :
        question_text, sol_text = generate_question(states[idquestion], capitals, NOPTIONS)
        quizzfile.write(question_text)
        solfile.write(sol_text)
    
    quizzfile.close()
    solfile.close()
    

Generating quizz number : 0
Generating quizz number : 1
Generating quizz number : 2
Generating quizz number : 3
Generating quizz number : 4
Generating quizz number : 5
Generating quizz number : 6
Generating quizz number : 7
Generating quizz number : 8
Generating quizz number : 9


# Chapter 12: Processing excel files

In [7]:
import openpyxl
wb = openpyxl.load_workbook('automate_online-materials/censuspopdata.xlsx')

sheet = wb['Population by Census Tract'] 
nrows = sheet.max_row
ncolumns = sheet.max_column

county_count = {} # use a dictionary
county_pop = {} # use a dictionary
print("Reading rows ...")
for irow in range(2, nrows+1):
    state  = sheet['B'+str(irow)].value
    county = sheet['C'+str(irow)].value
    popu   = sheet['D'+str(irow)].value
    # set default values
    county_count.setdefault(county, 0)
    county_pop.setdefault(county, 0)
    # Count
    county_count[county] += 1
    county_pop[county] += int(popu)

# write data to an excel file
print("Printing data ...")
ofile = openpyxl.Workbook()
sheet = ofile.active
sheet['A1'] = "County"
sheet['B1'] = "Count"
sheet['C1'] = "Population"
i = 2
for county in county_count.keys() :
    sheet['A'+str(i)] = county
    sheet['B'+str(i)] = county_count[county]
    sheet['C'+str(i)] = county_pop[county]
    i += 1
ofile.save('test/county_data.xlsx')

print("Done.")

Reading rows ...
Printing data ...
Done.


# Chapter 18 : Sending emails

In [15]:
import smtplib
import getpass # to read the password without echoing it
import openpyxl

# send email helper
def send_emails(smtpobj, fname, fromaddress):
    wb = openpyxl.load_workbook(fname)
    sheet = wb.active 
    nrows = sheet.max_row
    ncols = sheet.max_column
    # Create header for grades table
    header=""
    for icol in range(3, ncols+1):
        header += """{0:>10s}""".format(sheet.cell(1, icol).value)
    header += "\n"
    # Process each student grade
    for irow in range(2, nrows + 1):
        msg = """
        Dear student {0},
        These are your grades.\n""".format(sheet.cell(irow, 1).value)
        msg += header
        for icol in range(3, ncols+1):
            msg += """{0:>10d}""".format(sheet.cell(irow, icol).value)
        msg += "\n"
        msg += """Best regards,\n William"""
        smtpobj.sendmail(fromaddress, sheet.cell(irow, 2).value, 'Subject: Grades.\n' + msg)
    
# smtp stuff
SERVER='smtp.gmail.com'
PORT=587
USERNAME='woquendo@gmail.com'
PASSWORD=getpass.getpass()
FROM=USERNAME
TO=USERNAME # This will change
smtpObj = smtplib.SMTP(SERVER, PORT)
smtpObj.starttls()
smtpObj.login(USERNAME, PASSWORD)
send_emails(smtpObj, 'grades.xlsx', USERNAME)
smtpObj.quit()


        Dear student Alice,
        These are your grades.
       gr1       gr2       gr3
         1         2         3
Best regards,
 William

        Dear student Bob,
        These are your grades.
       gr1       gr2       gr3
         3         2         1
Best regards,
 William

        Dear student John,
        These are your grades.
       gr1       gr2       gr3
         4         5         6
Best regards,
 William

        Dear student Aegon,
        These are your grades.
       gr1       gr2       gr3
         6         5         4
Best regards,
 William


(221, b'2.0.0 closing connection q187-v6sm6156418vkq.10 - gsmtp')